# import 

In [25]:
import cv2

In [26]:
import random

In [ ]:
from fastai.vision import *

In [13]:
import os

# functions

In [55]:
# export
#选择一个类别的
def pick_mask_1c(img, color_val, dbg = []):
    '''
    根据数据集的mask图像，数一下图像上某个灰度值的点的数量。
    '''    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #先去掉高于这个值的
    thresh = cv2.threshold(gray, color_val, color_val, cv2.THRESH_TOZERO_INV)[1]
    #然后去掉低于这个值的
    thresh2 = cv2.threshold(thresh, color_val - 1, color_val, cv2.THRESH_BINARY)[1]
    if 'show_c' in dbg:
        cv2.imshow('pickmask_thres', thresh)
        cv2.imshow('pickmask_thres2', thresh2)
        #cv2.waitKey()
    dcnt = cv2.countNonZero(thresh2)
    return dcnt, thresh2

In [109]:
#对于CCF中的3.png，只有一块水体，导致在图上生成的所有有效图像都是围绕一个小水坑的。
#所以改用gen_dataset2
def gen_dataset(imgfns, maskfns, output_path = 'label/dataset_20200708', img_sz = (512, 512)
                , ds_sz = 10, blank_pct = 0.2, suffix = '.png', dbg = []):
    '''
    生成数据集。按照原始大图的数量平均分配每个生成的图片的数量。
    参数：
        imgfns：大图片文件名列表
        maskfns：对应的mask文件名列表。与上一个参数里面的文件一一对应
        output_path：输出文件目录
        img_sz：提取的小图的大小,内容为(width, height)
        ds_sz：生成图片数量
        blank_pct：无水体图片数量比例
        suffix：生成图片文件扩展名
        dbg：调试用
    '''
    assert len(imgfns) == len(maskfns)
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    #总的空白的数量。在总数比较少的情况下会产出为0，专门处理一下。
    TOTAL_BLANK_CNT = int(ds_sz * blank_pct)
    total_cnt = 0#产出计数
    total_cnt_blank = 0#总得空白的计数
    for imgfn, maskfn in zip(imgfns, maskfns):
        img = cv2.imread(imgfn)
        mask = cv2.imread(maskfn) 
        #print(imgfn, maskfn, img.shape, mask.shape)
        assert img.shape == mask.shape
                
        #当前大图需要产生的总的数据图片的数量
        DS_SZ_1IMG = ds_sz // len(imgfns)
        #当前大图需要产生的空白数据图片的数量
        DS_SZ_1IMG_BLANK = int(DS_SZ_1IMG * blank_pct)
        #比如总共产出10个数据集，0.2空白，三个大图，会导致每个图的空白数量都是0.这里专门处理一下
        if DS_SZ_1IMG_BLANK == 0 and TOTAL_BLANK_CNT > 0 and total_cnt_blank < TOTAL_BLANK_CNT:
            DS_SZ_1IMG_BLANK = 1
        #需要产生的有效图片的数量
        DS_SZ_1IMG_VALID = DS_SZ_1IMG - DS_SZ_1IMG_BLANK
        
        validcnt = 0 #当前图片产生的数量
        blankcnt = 0#当前产出的空白数据集的数量
        #print('dbg1', DS_SZ_1IMG, DS_SZ_1IMG_BLANK, DS_SZ_1IMG_VALID)
        #保证能凑成整数。而不是要求1000只有999
        while (total_cnt < ds_sz and total_cnt >= ds_sz - ds_sz % len(imgfns)) \
                or validcnt < DS_SZ_1IMG_VALID or blankcnt < DS_SZ_1IMG_BLANK:
            
            dsx = random.randint(0, img.shape[0] - img_sz[1])
            dsy = random.randint(0, img.shape[1] - img_sz[0])
            dsimg = img[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]

            dsmask = mask[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]
            rdsmask = dsmask.copy()
            
            area, dsmask = pick_mask_1c(dsmask, TYP_WATER)
            #根据比例判断一下是否有效的
            if area / (img_sz[0] * img_sz[1]) > BLANK_TH:
                print(validcnt, total_cnt, ds_sz % len(imgfns))
                #超过了数量的放弃.并且保证到达整数
                if validcnt >= DS_SZ_1IMG_VALID \
                        and total_cnt < ds_sz - ds_sz % len(imgfns): #凑整数
                    continue
                validcnt += 1
            else:
                if blankcnt >= DS_SZ_1IMG_BLANK:
                    continue
                blankcnt += 1
                total_cnt_blank += 1
            
            dsmask = dsmask * 255 #转换到白色
            
            rdsmask *= 60
            
            cv2.imwrite(os.path.join(output_path, '%05d.%s' % (total_cnt, suffix.split('.')[-1])), dsimg)
            cv2.imwrite(os.path.join(output_path, '%05d_mask.%s' % (total_cnt, suffix.split('.')[-1])), dsmask)
            if 'save_rmask' in dbg:
                cv2.imwrite(os.path.join(output_path, '%05d_mask_r.%s' % (total_cnt, suffix.split('.')[-1])), rdsmask)
            
            total_cnt += 1
           

In [128]:
#export
def gen_dataset2(imgfns, maskfns, output_dir = 'label/dataset_20200708', imgdir = 'image'
                , maskdir = 'mask', img_sz = (512, 512), ds_sz = 1000, blank_pct = 0.2
                , suffix = '.png', dbg = []):
    '''
    生成数据集
    参数：
        imgfns：大图片文件名列表
        maskfns：对应的mask文件名列表。与上一个参数里面的文件一一对应
        output_dir：输出数据集目录
        imgdir：输出数据集中存放图片的子目录。
        maskdir：输出数据集中存放mask图片的子目录
        img_sz：提取的小图的大小,内容为(width, height)
        ds_sz：生成图片数量
        blank_pct：无水体图片数量比例
        suffix：生成图片文件扩展名
        dbg：调试用
    '''
    assert len(imgfns) == len(maskfns)
    #检查创建目录
    for sdir in ['', imgdir, maskdir]:
        if not os.path.exists(os.path.join(output_dir, sdir)):
            os.makedirs(os.path.join(output_dir, sdir))
    total_cnt = 0#产出计数
    imgs = []
    for imgfn, maskfn in zip(imgfns, maskfns):
        img = cv2.imread(imgfn)
        mask = cv2.imread(maskfn) 
        #print(imgfn, maskfn, img.shape, mask.shape)
        assert img.shape == mask.shape
        imgs += [(img, mask)]
    
    #当前大图需要产生的空白数据图片的数量
    DS_SZ_BLANK = int(ds_sz * blank_pct)        
    #需要产生的有效图片的数量
    DS_SZ_VALID = ds_sz - DS_SZ_BLANK

    validcnt = 0 #当前图片产生的数量
    blankcnt = 0#当前产出的空白数据集的数量
    #print('dbg1', DS_SZ_1IMG, DS_SZ_1IMG_BLANK, DS_SZ_1IMG_VALID)
    #保证能凑成整数。而不是要求1000只有999
    while validcnt < DS_SZ_VALID or blankcnt < DS_SZ_BLANK:
        idx = random.randint(0, 2)
        #print(idx, imgfns[idx])
        img, mask = imgs[idx]
        dsx = random.randint(0, img.shape[0] - img_sz[1])
        dsy = random.randint(0, img.shape[1] - img_sz[0])
        dsimg = img[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]

        dsmask = mask[dsx : dsx + img_sz[1], dsy : dsy + img_sz[0]]
        rdsmask = dsmask.copy()

        area, dsmask = pick_mask_1c(dsmask, TYP_WATER)
        #根据比例判断一下是否有效的
        if area / (img_sz[0] * img_sz[1]) > BLANK_TH:
            #print(validcnt, total_cnt, ds_sz % len(imgfns))
            #超过了数量的放弃.并且保证到达整数
            if validcnt >= DS_SZ_VALID:
                continue
            validcnt += 1
        else:
            if blankcnt >= DS_SZ_BLANK:
                continue
            blankcnt += 1

        dsmask = dsmask * 85 #转换到白色

        rdsmask *= 60

        cv2.imwrite(os.path.join(output_dir, imgdir, '%05d.%s' % (total_cnt, suffix.split('.')[-1])), dsimg)
        cv2.imwrite(os.path.join(output_dir, maskdir, '%05d_mask.%s' % (total_cnt, suffix.split('.')[-1])), dsmask)
        if 'save_rmask' in dbg:
            cv2.imwrite(os.path.join(output_dir, maskdir, '%05d_mask_r.%s' % (total_cnt, suffix.split('.')[-1])), rdsmask)

        total_cnt += 1


# test

In [116]:
# export 
#原始的mask图像数值在0-4之间，这里放大一下到肉眼可以分辨的程度。
COLOR_STEP = 60

In [117]:
# export
# 关注的水体的灰度值。
#0 未标注。 1 空地。 2 建筑 。3 水体。4 道路
TYP_WATER = 3
SEG_WATER = TYP_WATER * COLOR_STEP

In [118]:
# export
#水体占比在这个值一下的认为是空的
BLANK_TH = 0.04

In [124]:
imgfns = ['data/src/image/1.png', 'data/src/image/2.png', 'data/src/image/3.png']

In [125]:
maskimgfns = ['data/label/train1_labels_8bits.png', 'data/label/train2_labels_8bits.png'
             , 'data/label/train3_labels_8bits.png']

In [121]:
#gen_dataset(imgfns, maskimgfns, ds_sz = 2000) 

In [129]:
gen_dataset2(imgfns, maskimgfns, ds_sz = 200, suffix='jpg') 

In [102]:
def pickds(imgfn, maskimgfn, SZ, dbg = []):
    '''
    从一个图像里面提取数据集
    输入:
        imgfn：要处理的大图
        maskimgfn：对应的mask图片
    '''
    ZOOM = 3
    img = cv2.imread(imgfn)
    mskimg = cv2.imread(maskimgfn)

    if 'show_c' in dbg:
        show_mskimg = cv2.resize(mskimg, (mskimg.shape[1] // ZOOM, mskimg.shape[0] // ZOOM))
        show_mskimg = show_mskimg * COLOR_STEP
        cv2.imshow('mask', show_mskimg)
        show_img = cv2.resize(img, (img.shape[1] // ZOOM, img.shape[0] // ZOOM))
        cv2.imshow('', show_img)

    r = do_pick(img, mskimg, SZ, dbg)
    
    if 'show_c' in dbg:
        cv2.waitKey()

In [101]:
def do_pick(img, mskimg, SZ, dbg):
    '''
    从原始图像里面随机提起一个图像，大小由SZ参数指定。
    '''
    dsx = random.randint(0, img.shape[0] - SZ[1])
    dsy = random.randint(0, img.shape[1] - SZ[0])
    dsimg = img[dsx : dsx + SZ[1], dsy : dsy + SZ[0]]
    
    dsmskimg = mskimg[dsx : dsx + SZ[1], dsy : dsy + SZ[0]]
    dsmskimg = dsmskimg * COLOR_STEP

    if 'show_c' in dbg:
        cv2.imshow('dsimg', dsimg)
        cv2.imshow('dmsksimg', dsmskimg)

    cnt, _ = pick_mask_1c(dsmskimg, SEG_WATER, dbg)

    if cnt / (SZ[0] * SZ[1]) < 0.04:
        #占比太小了。略过
        print('ignore')
        return False

    print(cnt, SZ[0] * SZ[1], dbg)
    return True

# export

In [130]:
!python notebook2script.py --fname 'gen_ds.ipynb' --outputDir './exp/'

Traceback (most recent call last):
  File "notebook2script.py", line 3, in <module>
    import json,fire,re
ModuleNotFoundError: No module named 'fire'
